# exp059
[Notion](https://www.notion.so/exp059-e5c187e4a903439eba146db5b394252e?pvs=4)  
ian pan先生の作成したextravasationデータのbounding boxの解析を行う.  

In [1]:
import os
import sys
from typing import Tuple, List, Dict, Any, Optional
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import warnings
import pydicom
import nibabel as nib

%matplotlib inline
rc('animation', html='jshtml')
matplotlib.rcParams['animation.embed_limit'] = 70
warnings.filterwarnings("ignore")

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')

from src.data_io import load_dicom_series
from src.image_processing import windowing
from src.visualization import view_image, print_injury, animate, apply_colormap_to_multilabel_images

In [2]:
df_train = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train.csv')
df_train_image_level = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/image_level_labels.csv')
df_train_series_meta = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')
bbox_df = pd.read_csv('data/ian-pan-ds/active_extravasation_bounding_boxes.csv')
base_dir = "data/rsna-2023-abdominal-trauma-detection"
dataset_dir = "data/dataset001"

In [3]:
bbox_df.head()

,filename,x1,y1,x2,y2,pid,series_id,instance_number,width,height
0,54371/55647/96.png,175,130,213,172,54371,55647,96,38,42
1,54371/55647/94.png,187,137,223,174,54371,55647,94,36,37
2,54371/55647/95.png,187,138,216,170,54371,55647,95,29,32
3,54371/55647/93.png,193,145,218,171,54371,55647,93,25,26
4,54371/55647/97.png,172,130,211,171,54371,55647,97,39,41


In [4]:
def load_series_from_dataset(dir_: str)-> Tuple[np.ndarray, list]:
    """seriesを読み込む.
    Args:
        dir_ (str): seriesのディレクトリ.
    Returns:
        np.ndarray: (Z, H, W)形式の画像の配列.
        list: 画像IDのリスト.
    """
    path_list = os.listdir(dir_)
    path_list = [[int(path.replace(".npy","")), path] for path in path_list]
    path_list.sort()
    image_id_list = [path[0] for path in path_list]
    path_list = [path[1] for path in path_list]
    arr = []
    for path in path_list:
        arr.append(np.load(os.path.join(dir_, path)))
    return np.array(arr), image_id_list

In [5]:
def apply_bbox_to_image(image: np.ndarray, x1: int, y1: int, x2: int, y2: int)-> np.ndarray:
    """画像にbounding boxを描画する."""
    y2 += 1
    x2 += 1
    image[y1:y2, x1:x1+2, :2] = 0
    image[y1:y2, x1:x1+2, 2] = 255
    image[y1:y1+2, x1:x2, :2] = 0
    image[y1:y1+2, x1:x2, 2] = 255
    image[y2-2:y2, x1:x2, :2] = 0
    image[y2-2:y2, x1:x2, 2] = 255
    image[y1:y2, x2-2:x2, :2] = 0
    image[y1:y2, x2-2:x2, 2] = 255
    return image
def add_bbox_marker(pid: int, sid: int, images: np.ndarray, image_id_list: list[int])-> np.ndarray:
    """画像中にbounding boxを描画する.
    Args:
        pid (int): 患者ID.
        sid (int): series ID.
        images (np.ndarray): (Z, H, W)形式の画像の配列. uint8で、ウィンドウ処理を終えていること.
        image_id_list (list[int]): 画像IDのリスト.
    """
    bbox_df_ = bbox_df[(bbox_df["pid"]==pid) & (bbox_df["series_id"]==sid)]
    if len(bbox_df_) == 0:
        return images
    bbox_df_ = bbox_df_.reset_index(drop=True)
    for i in range(len(bbox_df_)):
        image_id = bbox_df_.loc[i, "instance_number"]
        idx = image_id_list.index(image_id)
        image = images[idx]
        x1 = bbox_df_.loc[i, "x1"]
        y1 = bbox_df_.loc[i, "y1"]
        x2 = bbox_df_.loc[i, "x2"]
        y2 = bbox_df_.loc[i, "y2"]
        image = apply_bbox_to_image(image, x1, y1, x2, y2)
        images[idx] = image
    return images

In [6]:
df_train[df_train["extravasation_injury"] ==1]

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
16,10217,1,0,0,1,1,0,0,0,1,0,0,0,1,1
22,10292,1,0,0,1,1,0,0,1,0,0,1,0,0,1
28,10494,1,0,0,1,1,0,0,1,0,0,1,0,0,1
57,11044,1,0,0,1,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,7944,1,0,0,1,1,0,0,1,0,0,1,0,0,1
3060,820,0,1,0,1,1,0,0,1,0,0,1,0,0,1
3063,8263,1,0,0,1,0,1,0,0,0,1,1,0,0,1
3122,9528,1,0,0,1,1,0,0,1,0,0,1,0,0,1


In [10]:
df_train_series_meta[df_train_series_meta["patient_id"]==10292]

,patient_id,series_id,aortic_hu,incomplete_organ
32,10292,14945,116.0,0


In [13]:
pid, sid = 10292, 14945
series_dir = os.path.join(dataset_dir,'train_images', str(pid), str(sid))
imgs, image_id_list = load_series_from_dataset(series_dir)

In [14]:
# 大きすぎる場合この時点で間引く
imgs.shape

(89, 512, 512)

In [15]:
imgs = windowing(imgs, wl = 0, ww = 400)
# 1ch -> 3ch
imgs = np.stack((imgs,)*3, axis=-1)

In [16]:
bbox_added_imgs = add_bbox_marker(pid, sid, imgs, image_id_list)

In [ ]:
animate(bbox_added_imgs)